In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, root_mean_squared_error
from sklearn.neural_network import MLPRegressor
import optuna
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

num_trials = 100;

# Load data
shear_strength = pd.read_csv('shear_strength.csv', header=None)
new_or_old = pd.read_csv('new_or_old.csv', header=None)
load_type = pd.read_csv('load_type.csv', header=None)
wall_l = pd.read_csv('wall_l.csv', header=None)
wall_h = pd.read_csv('wall_h.csv', header=None)
wall_t = pd.read_csv('wall_t.csv', header=None)
leaf_num = pd.read_csv('leaf_num.csv', header=None)
bond_pattern = pd.read_csv('bond_pattern.csv', header=None)
ft_mortar = pd.read_csv('ft_mortar.csv', header=None)
ft_brick = pd.read_csv('ft_brick.csv', header=None)

# Combine features and target
X = pd.concat([new_or_old, wall_l, wall_t, leaf_num, bond_pattern, ft_mortar, ft_brick], axis=1)
X.columns = ['new_or_old', 'wall_l', 'wall_t', 'leaf_num', 'bond_pattern', 'ft_mortar', 'ft_brick']
y_raw = shear_strength 
y_raw.columns = ['shear_strength']
y = np.log(y_raw)  # log-transform (natural log)
y.columns = ['shear_strength']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=4)

# Scale data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

trial_results = []

# Optuna objective function
def objective(trial):
    n_layers = trial.suggest_int("n_layers", 1, 4) 
    hidden_layer_sizes = tuple(
        [trial.suggest_int(f"n_units_{i}", 100, 1000) for i in range(n_layers)] 
    )
    activation = trial.suggest_categorical("activation", ["relu", "tanh", "logistic"])

    model = MLPRegressor(
        hidden_layer_sizes=hidden_layer_sizes,
        activation=activation,
        max_iter=1000,
        random_state=0
    )

    cv = KFold(n_splits=10, shuffle=True, random_state=0)
    scores = cross_val_score(model, X_train, y_train.values.ravel(), cv=cv, scoring="neg_mean_squared_error")
    return -scores.mean()

# Run Optuna optimization
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=num_trials)

# Best parameters
print("Best parameters found by Optuna:", study.best_params)

trial_results = sorted(
    [(t.number, t.value, t.params) for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE],
    key=lambda x: x[1]
)[:10]

print("\nTop 10 Hyperparameter Combinations and Scores:")
for rank, (trial_number, mse, params) in enumerate(trial_results, start=1):
    model = MLPRegressor(
        hidden_layer_sizes=tuple([params[f"n_units_{i}"] for i in range(params["n_layers"])]),
        activation=params["activation"],
        max_iter=1000,
        random_state=0
    )
    
    model.fit(X_train, y_train.values.ravel())

    y_train_pred_log = model.predict(X_train)
    y_test_pred_log = model.predict(X_test)

    y_train_pred = np.exp(y_train_pred_log)
    y_test_pred = np.exp(y_test_pred_log)

    y_train_np = y_raw.iloc[y_train.index].to_numpy().squeeze()
    y_test_np = y_raw.iloc[y_test.index].to_numpy().squeeze()
    
    # Calculate metrics for training set
    r2_train = r2_score(y_train_np, y_train_pred)
    rmse_train = root_mean_squared_error(y_train_np, y_train_pred)
    ratio_train = y_train_np / y_train_pred
    mean_ratio_train = np.mean(ratio_train)
    cov_ratio_p_train = np.cov(ratio_train, rowvar=False, ddof=1)
    cov_ratio_p_train = cov_ratio_p_train * 100
    
    # Calculate metrics for testing set
    r2_test = r2_score(y_test_np, y_test_pred)
    rmse_test = root_mean_squared_error(y_test_np, y_test_pred)
    ratio_test = y_test_np / y_test_pred
    mean_ratio_test = np.mean(ratio_test)
    cov_ratio_p_test = np.cov(ratio_test, rowvar=False, ddof=1)
    cov_ratio_p_test = cov_ratio_p_test * 100
    
    # Print results
    print(f"Rank {rank}: Trial {trial_number}")
    print(f"Parameters: {params}")
    print(f"Training Set - R²: {r2_train:.3f}, RMSE: {rmse_train:.3f}, mean_ratio: {mean_ratio_train:.3f}, cov_ratio(%): {cov_ratio_p_train:.1f}")
    print(f"Testing Set  - R²: {r2_test:.3f}, RMSE: {rmse_test:.3f}, mean_ratio: {mean_ratio_test:.3f}, cov_ratio(%): {cov_ratio_p_test:.1f}\n")
    

[I 2025-04-14 10:27:50,903] A new study created in memory with name: no-name-098ded02-87b4-495c-ac38-bfbf0e752aed
[I 2025-04-14 10:33:42,035] Trial 0 finished with value: 0.7745851707288475 and parameters: {'n_layers': 3, 'n_units_0': 235, 'n_units_1': 964, 'n_units_2': 932, 'activation': 'tanh'}. Best is trial 0 with value: 0.7745851707288475.
[I 2025-04-14 10:37:15,783] Trial 1 finished with value: 1.7046949399032318 and parameters: {'n_layers': 3, 'n_units_0': 271, 'n_units_1': 845, 'n_units_2': 527, 'activation': 'tanh'}. Best is trial 0 with value: 0.7745851707288475.
[I 2025-04-14 10:39:14,112] Trial 2 finished with value: 0.5783377026404096 and parameters: {'n_layers': 2, 'n_units_0': 774, 'n_units_1': 915, 'activation': 'relu'}. Best is trial 2 with value: 0.5783377026404096.
[I 2025-04-14 10:40:39,369] Trial 3 finished with value: 0.5106866972037014 and parameters: {'n_layers': 2, 'n_units_0': 555, 'n_units_1': 953, 'activation': 'relu'}. Best is trial 3 with value: 0.51068669

Best parameters found by Optuna: {'n_layers': 4, 'n_units_0': 110, 'n_units_1': 641, 'n_units_2': 349, 'n_units_3': 519, 'activation': 'relu'}

Top 10 Hyperparameter Combinations and Scores:
Rank 1: Trial 14
Parameters: {'n_layers': 4, 'n_units_0': 110, 'n_units_1': 641, 'n_units_2': 349, 'n_units_3': 519, 'activation': 'relu'}
Training Set - R²: 0.949, RMSE: 0.098, mean_ratio: 1.009, cov_ratio(%): 1.6
Testing Set  - R²: 0.848, RMSE: 0.181, mean_ratio: 1.092, cov_ratio(%): 25.7

Rank 2: Trial 42
Parameters: {'n_layers': 3, 'n_units_0': 243, 'n_units_1': 166, 'n_units_2': 988, 'activation': 'relu'}
Training Set - R²: 0.949, RMSE: 0.099, mean_ratio: 1.010, cov_ratio(%): 1.8
Testing Set  - R²: 0.772, RMSE: 0.221, mean_ratio: 1.032, cov_ratio(%): 24.0

Rank 3: Trial 15
Parameters: {'n_layers': 3, 'n_units_0': 110, 'n_units_1': 436, 'n_units_2': 406, 'activation': 'relu'}
Training Set - R²: 0.949, RMSE: 0.099, mean_ratio: 1.008, cov_ratio(%): 1.7
Testing Set  - R²: 0.869, RMSE: 0.168, mean_